In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
import os
from IPython.display import display_html
import marko

from taogpt.orchestrator import *
from taogpt.utils import *
from taogpt.llm_model import LangChainLLM
from taogpt.prompts import PromptDb
import taogpt.utils as utils

utils.enable_debugging(0)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
TEMPERATURE = 0.7

In [ ]:
with open(os.path.join(os.environ['HOME'], '.ssh', 'openai-zillow.json'), 'r') as f:
    credentials = json.load(f)
os.environ["OPENAI_API_KEY"] = credentials['key']
os.environ["OPENAI_API_BASE"] = credentials['url']
llm3_5 = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=TEMPERATURE)
llm4 = ChatOpenAI(model_name='gpt-4-32k', temperature=TEMPERATURE)

conversation = ConversationChain(llm=llm3_5)
conversation.predict(input="What's your model version?")

In [ ]:
llm3_5.model_name, llm4.model_name

In [ ]:
prompts = PromptDb.load_defaults()
logger = MarkdownLogger('logs/taogpt_log.md')
executor = Orchestrator(llm=LangChainLLM(llm4, logger=logger), prompts=prompts, markdown_logger=logger,
                        initial_expansion=1,
                        max_tree_branches=2,
                        max_tokens=10000,
                        check_final=True,
                        # smarter_llm=LangChainLLM(llm4, logger=logger),
                        )

experiment_name = 'example'
executor.start("""
Solve this Game of 24: 7, 6, 2, 1
""", analyze_first=True)

In [ ]:
# executor.resume(0)

In [ ]:
logger = MarkdownLogger(f'examples/{experiment_name}.final.md')
logger.log_conversation(executor.show_conversation_thread(with_extras=True))
logger.log(f"**total tokens**: {executor.llm.total_tokens}")

In [ ]:
with open(logger._log_path, 'r') as f:
    html_output = marko.convert(f.read())
    display_html(html_output, raw=True)
